# Homework 4: Gesture Recognition

In [111]:
from __future__ import print_function
import cv2
import argparse
import numpy as np
import pyautogui
import time

In [2]:
def show(window_name, image):
    image = cv2.resize(image, (image.shape[1] // 2, image.shape[0] // 2))
    cv2.imshow(window_name, image)

### Code from thresholding and contours demo


In [3]:
# Actions
def hello_world():
    pyautogui.write('Hello world!') 

def zoom_in():  
    pyautogui.hotkey('command', '+')  
  
def zoom_out():  
    pyautogui.hotkey('command', '-')  
  
# def RotateLeft():  
#     pyautogui.hotkey('ctrl', 'R', presses=3)  


In [149]:
# Check for actions based on gestures



def check_action(window_average):

    if window_average['finger'] == 2:
        hello_world()
    elif window_average['finger'] == 3:
        zoom_out()
    elif window_average['finger'] == 4:
        zoom_in()

        
def check_complex_action(window):
    #print(window['angle'])
    angle_delta = np.sum(np.convolve(window['angle'], [-1,-1,-1,0,1,1,1], 'valid'))
    area_delta = np.sum(np.convolve(window['area'], [-1,-1,-1,0,1,1,1], 'valid'))
    if area_delta > 1500:
#         print("SMALLER")
        pyautogui.hotkey('ctrl', 'down')  
        return True
    elif area_delta < -1500:
#         print("BIGGER")
        pyautogui.hotkey('ctrl', 'up') 
        return True
    elif angle_delta > 35:
#         print("Rot RIGHT")
        pyautogui.hotkey('ctrl', 'right')
        return True
    elif angle_delta < -35:
#         print("Rot LEFT")
        pyautogui.hotkey('ctrl', 'left')
        return True
    
    else:
        return False
        

In [151]:
max_value = 255
max_type = 4
max_binary_value = 255
trackbar_type = 'Type: \n 0: Binary \n 1: Binary Inverted \n 2: Truncate \n 3: To Zero \n 4: To Zero Inverted'
trackbar_value = 'Value'
trackbar_blur = 'Blur kernel size'
window_name = 'Gesture Recognition'
isColor = False

def nothing(x):
    pass
    
cam = cv2.VideoCapture(2)
cam.set(cv2.CAP_PROP_AUTO_EXPOSURE, 0.25) # 0.25 turns OFF auto exp # nitzan
cam.set(cv2.CAP_PROP_AUTO_WB, 0.25) # 0.25 turns OFF auto WB # nitzan
# cam.set(cv2.CAP_PROP_EXPOSURE, -1.0) # nitzan
# cam.set(cv2.CAP_PROP_ISO_SPEED, 6.0) # nitzan

#cam.set(cv2.CAP_PROP_BRIGHTNESS, 5.0)
cv2.namedWindow(window_name)
cv2.createTrackbar(trackbar_type, window_name , 3, max_type, nothing)
# Create Trackbar to choose Threshold value
cv2.createTrackbar(trackbar_value, window_name , 0, max_value, nothing)
# Call the function to initialize
cv2.createTrackbar(trackbar_blur, window_name , 1, 20, nothing)
# create switch for ON/OFF functionality
color_switch = 'Color'
cv2.createTrackbar(color_switch, window_name,0,1,nothing)
cv2.createTrackbar('Contours', window_name,0,1,nothing)
cv2.createTrackbar('Lower Threshold', window_name, 0, 255, nothing)

window = {}
window_average = {'finger' : 0, 'x' : 0, 'y' : 0, 'MA' : 0, 'ma' : 0, 'angle' : 0}
window['finger'] = [0, 0, 0, 0, 0, 0, 0]
window['x'] = [0, 0, 0, 0, 0, 0, 0]
window['y'] = [0, 0, 0, 0, 0, 0, 0]
window['MA'] = [0, 0, 0, 0, 0, 0, 0]
window['ma'] = [0, 0, 0, 0, 0, 0, 0]
window['angle'] = [0, 0, 0, 0, 0, 0, 0]
window['area'] = [0] * 7 # NITZAN

prev_seconds = time.time()
while True:
    ret, frame = cam.read()
    if not ret:
        break
    
    #0: Binary
    #1: Binary Inverted
    #2: Threshold Truncated
    #3: Threshold to Zero
    #4: Threshold to Zero Inverted
    threshold_type = cv2.getTrackbarPos(trackbar_type, window_name)
    threshold_value = cv2.getTrackbarPos(trackbar_value, window_name)
    blur_value = cv2.getTrackbarPos(trackbar_blur, window_name)
    blur_value = blur_value+ (  blur_value%2==0)
    isColor = (cv2.getTrackbarPos(color_switch, window_name) == 1)
    findContours = (cv2.getTrackbarPos('Contours', window_name) == 1)
    lower_threshold = cv2.getTrackbarPos('Lower Threshold', window_name)

    # PART 0 START
    #show('fefe', frame)
    # PART 0 END
    
    # PART 1 START
#     lower_HSV = np.array([0, 40, 0], dtype = "uint8")  
#     upper_HSV = np.array([25, 255, 255], dtype = "uint8")  
    
    lower_HSV = np.array([81, 0, 63], dtype = "uint8") # NITZAN 
    upper_HSV = np.array([204, 124, 140], dtype = "uint8")  # NITZAN

    convertedHSV = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)  
    skinMaskHSV = cv2.inRange(convertedHSV, lower_HSV, upper_HSV)  


#     lower_YCrCb = np.array((0, 138, 67), dtype = "uint8")  
#     upper_YCrCb = np.array((255, 173, 133), dtype = "uint8")  
    
    lower_YCrCb = np.array((40, 138, 67), dtype = "uint8")  # NITZAN
    upper_YCrCb = np.array((255, 173, 133), dtype = "uint8")  # NITZAN

    convertedYCrCb = cv2.cvtColor(frame, cv2.COLOR_BGR2YCrCb)  
    skinMaskYCrCb = cv2.inRange(convertedYCrCb, lower_YCrCb, upper_YCrCb)  

    skinMask = cv2.add(skinMaskHSV,skinMaskYCrCb)  
    #show(window_name, skinMask)
        
    # Erosion and Dilation
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))  
    skinMask = cv2.erode(skinMask, kernel, iterations = 2)  
    skinMask = cv2.dilate(skinMask, kernel, iterations = 2)  
    # blur the mask to help remove noise, then apply the  
    # mask to the frame  
    skinMask = cv2.GaussianBlur(skinMask, (7, 7), 0) # NITZAN 9,9 --> 7,7
    skin = cv2.bitwise_and(frame, frame, mask = skinMask) 
#     show(window_name, skin)
    # PART 1 END
    
    # PART 2 START
    # Binarize the image
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  
    gray = cv2.cvtColor(skin, cv2.COLOR_BGR2GRAY)  # NITZAN
    
#     ret, thresh = cv2.threshold(gray, 0, max_binary_value, cv2.THRESH_BINARY+cv2.THRESH_OTSU )  # SHUBHA: USE WHEN RUNNING PART 2
    ret, thresh = cv2.threshold(gray, 0, max_binary_value, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU )  # NITZAN

#     show(window_name, thresh)
    
    # Show connected components
    ret, markers, stats, centroids = cv2.connectedComponentsWithStats(thresh,ltype=cv2.CV_16U)  
    markers = np.array(markers, dtype=np.uint8)  
    label_hue = np.uint8(179*markers/np.max(markers))  
    blank_ch = 255*np.ones_like(label_hue)  
    labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)
    labeled_img[label_hue==0] = 0  
#     show(window_name, labeled_img)

    statsSortedByArea = stats[np.argsort(stats[:, 4])]  
    
    if (ret>2):  
        try:  
            roi = statsSortedByArea[-3][0:4]  
            x, y, w, h = roi  
            subImg = labeled_img[y:y+h, x:x+w]  
            subImg = cv2.cvtColor(subImg, cv2.COLOR_BGR2GRAY);  
#             _, contours, _ = cv2.findContours(subImg, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            contours, _ = cv2.findContours(subImg, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)  # NITZAN
            maxCntLength = 0  
            for i in range(0,len(contours)):  
                cntLength = len(contours[i])  
                if(cntLength>maxCntLength):  
                    cnt = contours[i]  
                    maxCntLength = cntLength  
            if(maxCntLength>=5):  
                ellipseParam = cv2.fitEllipse(cnt)  
                subImg = cv2.cvtColor(subImg, cv2.COLOR_GRAY2RGB);  
                subImg = cv2.ellipse(subImg,ellipseParam,(0,255,0),2)  
              
            subImg = cv2.resize(subImg, (0,0), fx=3, fy=3)          
            (x,y),(MA,ma),angle = cv2.fitEllipse(cnt)
            
            # PART 4: WINDOW AVERAGING START
            window['x'].append(x)
            window['x'] = window['x'][1:]
            window_average['x'] = round(np.average(window['x']))
            
            window['y'].append(x)
            window['y'] = window['y'][1:]
            window_average['y'] = round(np.average(window['y']))
            
            window['MA'].append(MA)
            window['MA'] = window['MA'][1:]
            window_average['MA'] = round(np.average(window['MA']))
            
            window['ma'].append(ma)
            window['ma'] = window['ma'][1:]
            window_average['ma'] = round(np.average(window['ma']))
            
            window['angle'].append(angle)
            window['angle'] = window['angle'][1:]
            window_average['angle'] = round(np.average(window['angle']))
            
            window['area'].append(w*h)
            window['area'] = window['area'][1:]
            # PART 4: WINDOW AVERAGING END
 
#             cv2.imshow("ROI "+str(2), subImg)
#             print("Ellipse Center: " + str((x, y)) + ", Major & Minor Axis: " + str((MA, ma)) + ", Orientation: " + str(angle) + '\n')
            
            cv2.waitKey(1)  
        except:  
#             print("No hand found")
            pass
    # PART 2 END

#     # PART 3A START
    #ret, thresh = cv2.threshold(gray, 0, max_binary_value, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU )
    ret, thresh = cv2.threshold(gray, 0, max_binary_value, cv2.THRESH_BINARY+cv2.THRESH_OTSU )  # NITZAN
    thresholdedHandImage = thresh
    contours, _ = cv2.findContours(thresholdedHandImage, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)       
    contours = sorted(contours,key=cv2.contourArea,reverse=True)
    if len(contours)>1:  
        largestContour = contours[0]  
        hull = cv2.convexHull(largestContour, returnPoints = False)     
        for cnt in contours[:1]:  
            defects = cv2.convexityDefects(cnt,hull)  
            if(not isinstance(defects,type(None))):
                thresholdedHandImage = cv2.cvtColor(thresholdedHandImage, cv2.COLOR_GRAY2BGR)
                fingerCount = 1
                for i in range(defects.shape[0]):  
                    s,e,f,d = defects[i,0]  
                    start = tuple(cnt[s][0])  
                    end = tuple(cnt[e][0])  
                    far = tuple(cnt[f][0]) 
                    
                    # UNCOMMENT FOR PART 3A
                    # draws countour line around hand and red dots in the contour's defects
#                     thresholdedHandImage = cv2.line(thresholdedHandImage,start,end,[0,255,0],2)  
#                     thresholdedHandImage = cv2.circle(thresholdedHandImage,far,5,[0,0,255],-1)
#                     show(window_name, thresholdedHandImage)
                    # UNCOMMENT FOR PART 3A
                    
                    # PART 3B START
                    c_squared = (end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2  
                    a_squared = (far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2  
                    b_squared = (end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2  
                    angle = np.arccos((a_squared + b_squared  - c_squared ) / (2 * np.sqrt(a_squared * b_squared )))    

                    if angle <= np.pi / 3:  
                        fingerCount += 1  
                        cv2.circle(thresholdedHandImage, far, 10, [0, 0, 255], -1)
#                         thresholdedHandImage = cv2.line(thresholdedHandImage,start,end,[0,255,0],2)  
#                         thresholdedHandImage = cv2.circle(thresholdedHandImage,far,5,[0,0,255],-1)
#                     show(window_name, thresholdedHandImage)
    
                # Write window average and finger count into the image start
                # PART 4: WINDOW AVERAGING START
                window['finger'].append(fingerCount)
                window['finger'] = window['finger'][1:]
                window_average['finger'] = round(np.average(window['finger']))
                # PART 4: WINDOW AVERAGING END

                font = cv2.FONT_HERSHEY_SIMPLEX
                height, width, _ = thresholdedHandImage.shape
                bottomLeftCornerOfText1 = (int(width * 0.01), int(height * .92)) # NITZAN
                bottomLeftCornerOfText2 = (int(width * 0.01), int(height * .98)) # NITZAN
                fontScale = 1
                fontColor = (255,255,255)
                lineType = 2
                
                cv2.putText(thresholdedHandImage, "Finger Count: " + str(fingerCount), bottomLeftCornerOfText1, font, fontScale, fontColor, lineType)
                cv2.putText(thresholdedHandImage, "Window Average: " + str(window_average['finger']), bottomLeftCornerOfText2, font, fontScale, fontColor, lineType)
                #Write window average and finger count into the image end
    
                show(window_name, thresholdedHandImage)
#                 # PART 3B END  
#     # PART 3A END
    
#     # PART 4 START
    #check_action(window_average)
    
    # don't check for action again for 0.5 sec to limit num of commands per second
    curr_seconds = time.time() # get curr time
    act_ret = False
    if (curr_seconds - prev_seconds) > 1.: # if at least 0.5 sec passed then check action
        act_ret = check_complex_action(window)
        if act_ret: # if action found, reset timer
            prev_seconds = time.time()
#     # PART 4 END
    
    k = cv2.waitKey(1) #k is the key pressed
    if k == 27 or k==113:  #27, 113 are ascii for escape and q respectively
        #exit
        cv2.destroyAllWindows()
        cam.release()
        break